# Generalized Class for Solving Multidimensional Linear Systems

### Note: Define all needed symbols outside the class

In this file we write a solver for an arbitrary linear system and show it works for a 3 dimensional example system

In [1]:
from sympy import *
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
class SystemSolve():
    def __init__(self,dim,terms,A,deriv,print_on=True):
        self.dim = dim
        self.terms = terms
        self.A = A
        self.deriv = deriv
        self._createB()
        self.p = print_on
        return
    
    def _createB(self):
        self.B = []
        for i in range(self.terms):
            b = []
            for j in range(1,self.dim+1):
                b.append(sympify('B{}_{}'.format(i,j)))
            self.B.append(b)
        return
    
    def setEigs(self, e_val, e_vec):
        self.e_val = e_val
        self.B_tups = []
        b_tups = []
        for i in range(self.dim):
            b_tups.append((self.B[0][i],e_vec[i]))

        self.B_tups.append(b_tups)
        return
    
    def createSys(self):
        W = []
        W_deriv = []

        for j in range(self.dim):
            w = sympify(0)
            for i in range(self.terms):
                w += self.B[i][j]*sigma**i
            w_deriv = diff(w, sigma)

            W.append(w)
            W_deriv.append(w_deriv)

        # Store W and W' as numpy arrays
        self.W = np.array(W)
        self.W_deriv = np.array(W_deriv)

        # Replace matrix A by A-lam*I
        Anew = self.A - np.eye(self.dim)*alpha

        # Define system of equations from each row of A
        eqs = []
        for i in range(self.dim):
            eq = np.dot(Anew[i],self.W)-self.W_deriv[i]*self.deriv
            eqs.append(eq)
   

        # Isolate coefficients for each power of sigma
        eq_der = [eq.copy() for eq in eqs]
        heqs = []

        for i in range(self.terms):
            if i==0: prod = 1
            else: 
                prod *= i

            if self.p: print('\n\npower = {}'.format(i)) 
            res = [eqd.subs(sigma,0)/prod for eqd in eq_der]

            if self.p:
                for r in res:
                    print('\n')
                    print(r)

            heqs.append(res)    
            eq_der = [diff(eqd,sigma) for eqd in eq_der]

        # Solve for the coefficients of W

        up = 1
        stop = True
        for j in range(len(heqs)):
            if j == len(heqs)-1 and stop:
                break
            if self.p:
                p1 = '-'*21+'\nsystem when sigma = '
                p2 = str(j)+'\n'+'-'*21+'\n'
                print(p1+p2)
                
            if j == 0:
                try: 
                    matrix, vector = linear_eq_to_matrix(heqs[j],self.B[j+up])
                    soln = matrix.LUsolve(vector)
                except: 
                    #print('failed')
                    up = 0
                    stop = False
            matrix, vector = linear_eq_to_matrix(heqs[j],self.B[j+up])
            soln = matrix.LUsolve(vector)
            b_tups = []
            for i in range(self.dim):
                val = str(self.B[j+up][i]) +' = '+str(soln[i])
                if j != 0 or stop: 
                    b_tups.append((self.B[j+up][i],soln[i].subs(self.B_tups[-1])))
                if self.p: print(val,'\n\n')
            if j != 0 or stop: self.B_tups.append(b_tups)
                
        self.B_vals = [i for s in self.B_tups for i in s]
        
    def evalW(self, sig_val, lam_val=None, other_vals=[]):
        add_vals = [(alpha,self.e_val),(sigma,sig_val)]
        if lam_val is not None: add_vals.append((lam,lam_val))
            
        to_sub = self.B_vals + add_vals + other_vals
        W = [w.subs(to_sub) for w in self.W]
        return W
    
    def checkBNorms(self, lam_val, other_vals=[]):
        norms = []
        all_vals = self.B_vals+[(alpha,self.e_val),(lam,lam_val)]+other_vals
        for b in tqdm(self.B):
            norm = sympify(0)
            for i in b:
                norm = norm + Abs(i)
            norm = (norm)**(1/2)
            
            norm = norm.subs(all_vals)
            #print(norm)
            norms.append(norm)

        return norms

## Practice System

In [4]:
sigma, alpha, lam = symbols('sigma,alpha,lam')
dim = 3
num_terms = 10
A = np.array([[3+3*sigma,lam,1],[0,-sigma-1,lam],[0,0,-2-sigma]])
d_dx = -2*sigma-sigma**2
e_val = -1
e_vec = np.array([lam, -4, 0])

In [5]:
my_sys = SystemSolve(dim, num_terms, A, d_dx,False)
my_sys.setEigs(e_val,e_vec)
my_sys.createSys()

In [6]:
my_sys.evalW(.5,100)

[93.3333079020182, -5.00000000000000, 0]

In [7]:
my_sys.evalW(.3,100)

[95.6521737458985, -4.60000000000000, 0]

In [8]:
norms = my_sys.checkBNorms(100)

100%|██████████| 10/10 [00:01<00:00,  6.39it/s]


In [9]:
norms

[10.1980390271856,
 4.32049379893857,
 2.88675134594813,
 2.04124145231932,
 1.44337567297406,
 1.02062072615966,
 0.721687836487032,
 0.510310363079829,
 0.360843918243516,
 0.255155181539914]